<a href="https://colab.research.google.com/github/miyoko-shimura/coursera-building-systems-with-chatgptapi/blob/main/1_Building_Systems_with_the_ChatGPT_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L1 Language Models, the Chat Format and Tokens

## Setup
#### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [6]:
!pip install openai==0.28
!pip install tiktoken

import os
import openai
import tiktoken
from google.colab import userdata

# Retrieve the OpenAI API key from Colab's userdata
openai.api_key = userdata.get('OpenAI')

#### helper function
This may look familiar if you took the earlier course "ChatGPT Prompt Engineering for Developers" Course.

Throughout this course, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat).

This helper function will make it easier to use prompts and look at the generated outputs.

**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to produce the desired results.

In [9]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

**Note**: This and all other lab notebooks of this course use OpenAI library version `0.27.0`.

In order to use the OpenAI library version `1.0.0`, here is the code that you would use instead for the get_completion function:

```python
client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content
```

## Prompt the model and get a completion

In [11]:
response = get_completion("What is the capital of France?")

In [12]:
print(response)

The capital of France is Paris.


## Tokens

In [13]:
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)

pilpolol


"lollipop" in reverse should be "popillol"

In [14]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")

In [15]:
response

'p-o-p-i-l-l-o-l'

## Helper function (chat format)
Here's the helper function we'll use in this course.

In [18]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut
    )
    return response.choices[0].message["content"]

In [19]:
messages =  [
{'role':'system',
 'content':"""You are an assistant who\
 responds in the style of Dr Seuss."""},
{'role':'user',
 'content':"""write me a very short poem\
 about a happy carrot"""},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Oh, the happy carrot, orange and bright,
Grew in the garden, a joyful sight,
With a leafy green top, reaching so high,
Bathed in sunshine from the sky so nigh.
It sang a tune, so merry and fine,
"O happy day, this life of mine!"


In [20]:
# length
messages =  [
{'role':'system',
 'content':'All your responses must be \
one sentence long.'},
{'role':'user',
 'content':'write me a story about a happy carrot'},
]
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once upon a time, a cheerful carrot named Charlie grew big and proud in the sunny vegetable patch.


In [21]:
# combined
messages =  [
{'role':'system',
 'content':"""You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."""},
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
]
response = get_completion_from_messages(messages,
                                        temperature =1)
print(response)

In a field of green, a carrot with a grin, danced and sang with a joyful spin.


In [23]:
def get_completion_and_token_count(messages,
                                   model="gpt-3.5-turbo",
                                   temperature=0,
                                   max_tokens=500):

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    content = response.choices[0].message["content"]

    token_dict = {
'prompt_tokens':response['usage']['prompt_tokens'],
'completion_tokens':response['usage']['completion_tokens'],
'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

In [24]:
messages = [
{'role':'system',
 'content':"""You are an assistant who responds\
 in the style of Dr Seuss."""},
{'role':'user',
 'content':"""write me a very short poem \
 about a happy carrot"""},
]
response, token_dict = get_completion_and_token_count(messages)

In [25]:
print(response)

Oh, the happy carrot, so bright and so bold,
In the garden, its story is joyfully told.
With a leafy green top and a vibrant orange hue,
It dances and sings, bringing smiles to you.


In [26]:
print(token_dict)

{'prompt_tokens': 37, 'completion_tokens': 45, 'total_tokens': 82}


#### Notes on using the OpenAI API outside of this classroom

To install the OpenAI Python library:
```
!pip install openai
```

The library needs to be configured with your account's secret key, which is available on the [website](https://platform.openai.com/account/api-keys).

You can either set it as the `OPENAI_API_KEY` environment variable before using the library:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

Or, set `openai.api_key` to its value:

```
import openai
openai.api_key = "sk-..."
```

#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.